# 💰 Government Salary Database Scraper
### Built by Jimmy Tools | jimmytools.net

This notebook extracts public employee salary data from state government databases.

**What you'll get:**
- Employee names, titles, departments
- Base salary, overtime, total compensation
- Searchable by name, agency, or salary range
- Export to CSV/Excel

**Supported States:**
- 🌴 California (Transparent California)
- ⭐ Texas (Texas Tribune)
- 🌞 Florida (Florida Has A Right To Know)
- 🗽 New York (SeeThroughNY)
- 🌽 Illinois (Family Taxpayers Foundation)

---

## 🚀 Quick Start
1. Click **Runtime → Run all** (or press Ctrl+F9)
2. Choose a state and search criteria
3. Download your results!

---

In [ ]:
#@title 🔧 Setup (runs automatically)
import requests
import pandas as pd
import json
import re
import time
from datetime import datetime
from bs4 import BeautifulSoup
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/html, */*',
}

print("✅ Setup complete! Ready to search government salaries.")

In [ ]:
#@title 🗺️ Choose State

print("Which state do you want to search?\n")
print("1. California (Transparent California)")
print("2. Texas (Texas Tribune Salaries)")
print("3. Florida (Florida Has A Right To Know)")
print("4. New York (SeeThroughNY)")
print("5. Illinois (OpenTheBooks)")

state_choice = input("\nEnter number (1-5): ").strip()

state_map = {
    '1': ('california', 'California'),
    '2': ('texas', 'Texas'),
    '3': ('florida', 'Florida'),
    '4': ('newyork', 'New York'),
    '5': ('illinois', 'Illinois'),
}

state_code, state_name = state_map.get(state_choice, ('california', 'California'))
print(f"\n✅ Selected: {state_name}")

In [ ]:
#@title 🔍 Search Options

print("\nHow do you want to search?\n")
print("1. Search by name (e.g., 'John Smith')")
print("2. Search by agency/department (e.g., 'Police', 'Fire')")
print("3. Search by job title (e.g., 'Manager', 'Director')")
print("4. Top earners (highest salaries)")
print("5. Get all employees for an agency")

search_choice = input("\nEnter choice (1-5): ").strip()

search_term = ''
if search_choice in ['1', '2', '3']:
    prompts = {
        '1': 'Enter name to search: ',
        '2': 'Enter agency/department name: ',
        '3': 'Enter job title: ',
    }
    search_term = input(prompts[search_choice]).strip()
    print(f"\n🔍 Searching for: {search_term}")
elif search_choice == '4':
    print("\n📊 Will fetch top earners")
elif search_choice == '5':
    search_term = input("Enter exact agency name: ").strip()
    print(f"\n🏛️ Will fetch all employees from: {search_term}")

In [ ]:
#@title 📥 California - Transparent California

def search_california(search_type, search_term, limit=500):
    """
    Search Transparent California database
    https://transparentcalifornia.com
    """
    results = []
    base_url = 'https://transparentcalifornia.com/api/salaries/'
    
    # Build search URL based on type
    if search_type == '1':  # Name search
        url = f"{base_url}?name={requests.utils.quote(search_term)}&size={limit}"
    elif search_type == '2':  # Agency
        url = f"{base_url}?employer={requests.utils.quote(search_term)}&size={limit}"
    elif search_type == '3':  # Title
        url = f"{base_url}?job_title={requests.utils.quote(search_term)}&size={limit}"
    elif search_type == '4':  # Top earners
        url = f"{base_url}?sort=-total_pay&size={limit}"
    else:  # All from agency
        url = f"{base_url}?employer={requests.utils.quote(search_term)}&size={limit}"
    
    try:
        # Transparent California uses a public web interface
        # We'll scrape the search results page instead
        search_url = f"https://transparentcalifornia.com/salaries/search/?q={requests.utils.quote(search_term)}"
        response = requests.get(search_url, headers=HEADERS, timeout=30)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find salary table rows
        rows = soup.select('table.table tbody tr')
        
        for row in rows[:limit]:
            cols = row.find_all('td')
            if len(cols) >= 5:
                name_link = cols[0].find('a')
                results.append({
                    'name': name_link.text.strip() if name_link else cols[0].text.strip(),
                    'job_title': cols[1].text.strip(),
                    'employer': cols[2].text.strip(),
                    'total_pay': cols[3].text.strip(),
                    'year': cols[4].text.strip() if len(cols) > 4 else '',
                    'state': 'California',
                    'source': 'Transparent California'
                })
    except Exception as e:
        print(f"Error fetching California data: {e}")
    
    return pd.DataFrame(results)

if state_code == 'california':
    print(f"\n⏳ Searching California salaries...")
    df_salaries = search_california(search_choice, search_term)
    print(f"✅ Found {len(df_salaries)} records")

In [ ]:
#@title 📥 Texas - Texas Tribune Salaries

def search_texas(search_type, search_term, limit=500):
    """
    Search Texas Tribune Government Salaries Explorer
    https://salaries.texastribune.org
    """
    results = []
    
    try:
        # Texas Tribune has a searchable database
        search_url = f"https://salaries.texastribune.org/search/?q={requests.utils.quote(search_term)}"
        response = requests.get(search_url, headers=HEADERS, timeout=30)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Parse search results
        cards = soup.select('.search-result, .employee-card, tr.employee')
        
        for card in cards[:limit]:
            name = card.select_one('.name, .employee-name, td:first-child')
            title = card.select_one('.title, .job-title, td:nth-child(2)')
            agency = card.select_one('.agency, .employer, td:nth-child(3)')
            salary = card.select_one('.salary, .pay, td:nth-child(4)')
            
            if name:
                results.append({
                    'name': name.text.strip(),
                    'job_title': title.text.strip() if title else '',
                    'employer': agency.text.strip() if agency else '',
                    'total_pay': salary.text.strip() if salary else '',
                    'state': 'Texas',
                    'source': 'Texas Tribune'
                })
    except Exception as e:
        print(f"Note: Texas Tribune scraping may require updates. Error: {e}")
        print("Trying alternative approach...")
        
        # Alternative: Try the API endpoint if available
        try:
            api_url = f"https://salaries.texastribune.org/api/employees/?search={requests.utils.quote(search_term)}&limit={limit}"
            resp = requests.get(api_url, headers=HEADERS, timeout=30)
            if resp.status_code == 200:
                data = resp.json()
                for emp in data.get('results', data.get('employees', [])):
                    results.append({
                        'name': emp.get('name', ''),
                        'job_title': emp.get('title', emp.get('job_title', '')),
                        'employer': emp.get('agency', emp.get('employer', '')),
                        'total_pay': emp.get('salary', emp.get('total_pay', '')),
                        'state': 'Texas',
                        'source': 'Texas Tribune'
                    })
        except:
            pass
    
    return pd.DataFrame(results)

if state_code == 'texas':
    print(f"\n⏳ Searching Texas salaries...")
    df_salaries = search_texas(search_choice, search_term)
    print(f"✅ Found {len(df_salaries)} records")

In [ ]:
#@title 📥 Florida - Florida Has A Right To Know

def search_florida(search_type, search_term, limit=500):
    """
    Search Florida government salary data
    https://floridahasarighttoknow.myflorida.com
    """
    results = []
    
    try:
        # Florida's People First data
        search_url = f"https://floridahasarighttoknow.myflorida.com/search?q={requests.utils.quote(search_term)}"
        response = requests.get(search_url, headers=HEADERS, timeout=30)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        rows = soup.select('table tr, .result-row')
        
        for row in rows[:limit]:
            cols = row.find_all('td') or row.find_all('.col')
            if len(cols) >= 4:
                results.append({
                    'name': cols[0].text.strip(),
                    'job_title': cols[1].text.strip() if len(cols) > 1 else '',
                    'employer': cols[2].text.strip() if len(cols) > 2 else '',
                    'total_pay': cols[3].text.strip() if len(cols) > 3 else '',
                    'state': 'Florida',
                    'source': 'Florida Has A Right To Know'
                })
    except Exception as e:
        print(f"Error fetching Florida data: {e}")
    
    return pd.DataFrame(results)

if state_code == 'florida':
    print(f"\n⏳ Searching Florida salaries...")
    df_salaries = search_florida(search_choice, search_term)
    print(f"✅ Found {len(df_salaries)} records")

In [ ]:
#@title 📥 New York - SeeThroughNY

def search_newyork(search_type, search_term, limit=500):
    """
    Search New York government salary data
    https://seethroughny.net
    """
    results = []
    
    try:
        # SeeThroughNY payroll search
        search_url = f"https://www.seethroughny.net/payrolls"
        
        # Build search parameters
        params = {
            'search': search_term,
            'limit': limit
        }
        
        response = requests.get(search_url, params=params, headers=HEADERS, timeout=30)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Parse results table
        rows = soup.select('table tbody tr')
        
        for row in rows[:limit]:
            cols = row.find_all('td')
            if len(cols) >= 4:
                results.append({
                    'name': cols[0].text.strip(),
                    'employer': cols[1].text.strip() if len(cols) > 1 else '',
                    'job_title': cols[2].text.strip() if len(cols) > 2 else '',
                    'total_pay': cols[3].text.strip() if len(cols) > 3 else '',
                    'state': 'New York',
                    'source': 'SeeThroughNY'
                })
    except Exception as e:
        print(f"Error fetching New York data: {e}")
    
    return pd.DataFrame(results)

if state_code == 'newyork':
    print(f"\n⏳ Searching New York salaries...")
    df_salaries = search_newyork(search_choice, search_term)
    print(f"✅ Found {len(df_salaries)} records")

In [ ]:
#@title 📥 Illinois - OpenTheBooks

def search_illinois(search_type, search_term, limit=500):
    """
    Search Illinois government salary data
    https://www.openthebooks.com
    """
    results = []
    
    try:
        # OpenTheBooks API
        api_url = f"https://www.openthebooks.com/api/search"
        
        params = {
            'q': search_term,
            'state': 'IL',
            'limit': limit
        }
        
        response = requests.get(api_url, params=params, headers=HEADERS, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            for emp in data.get('results', []):
                results.append({
                    'name': emp.get('name', ''),
                    'job_title': emp.get('title', ''),
                    'employer': emp.get('employer', ''),
                    'total_pay': emp.get('salary', emp.get('pay', '')),
                    'state': 'Illinois',
                    'source': 'OpenTheBooks'
                })
        else:
            # Fallback: try web scraping
            search_url = f"https://www.openthebooks.com/search/?q={requests.utils.quote(search_term)}&state=IL"
            response = requests.get(search_url, headers=HEADERS, timeout=30)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            rows = soup.select('.search-result, table tr')
            for row in rows[:limit]:
                cols = row.find_all('td') or row.find_all('.field')
                if len(cols) >= 3:
                    results.append({
                        'name': cols[0].text.strip(),
                        'job_title': cols[1].text.strip() if len(cols) > 1 else '',
                        'employer': cols[2].text.strip() if len(cols) > 2 else '',
                        'total_pay': cols[3].text.strip() if len(cols) > 3 else '',
                        'state': 'Illinois',
                        'source': 'OpenTheBooks'
                    })
    except Exception as e:
        print(f"Error fetching Illinois data: {e}")
    
    return pd.DataFrame(results)

if state_code == 'illinois':
    print(f"\n⏳ Searching Illinois salaries...")
    df_salaries = search_illinois(search_choice, search_term)
    print(f"✅ Found {len(df_salaries)} records")

In [ ]:
#@title 👀 Preview Results

if 'df_salaries' in dir() and len(df_salaries) > 0:
    print(f"\n📊 {state_name} Government Salaries\n")
    print(f"Total records found: {len(df_salaries)}\n")
    
    # Clean up salary column for sorting
    def parse_salary(s):
        if pd.isna(s):
            return 0
        s = str(s).replace('$', '').replace(',', '').strip()
        try:
            return float(s)
        except:
            return 0
    
    df_salaries['salary_numeric'] = df_salaries['total_pay'].apply(parse_salary)
    df_display = df_salaries.sort_values('salary_numeric', ascending=False)
    
    print("Top 20 by salary:\n")
    print(df_display[['name', 'job_title', 'employer', 'total_pay']].head(20).to_string(index=False))
else:
    print("\n⚠️ No results found. Try a different search term or check the state selected.")

In [ ]:
#@title 🔍 Filter Results (Optional)

if 'df_salaries' in dir() and len(df_salaries) > 0:
    print("\nFilter options:")
    print("1. Filter by minimum salary")
    print("2. Filter by maximum salary")
    print("3. Filter by job title keyword")
    print("4. No filter (keep all)")
    
    filter_choice = input("\nEnter choice (1-4): ").strip()
    
    df_filtered = df_salaries.copy()
    
    if filter_choice == '1':
        min_sal = input("Enter minimum salary (e.g., 100000): ").strip()
        min_sal = float(min_sal.replace(',', '').replace('$', ''))
        df_filtered = df_filtered[df_filtered['salary_numeric'] >= min_sal]
        print(f"\n✅ Filtered to {len(df_filtered)} records with salary >= ${min_sal:,.0f}")
    elif filter_choice == '2':
        max_sal = input("Enter maximum salary (e.g., 50000): ").strip()
        max_sal = float(max_sal.replace(',', '').replace('$', ''))
        df_filtered = df_filtered[df_filtered['salary_numeric'] <= max_sal]
        print(f"\n✅ Filtered to {len(df_filtered)} records with salary <= ${max_sal:,.0f}")
    elif filter_choice == '3':
        keyword = input("Enter job title keyword: ").strip()
        df_filtered = df_filtered[df_filtered['job_title'].str.contains(keyword, case=False, na=False)]
        print(f"\n✅ Filtered to {len(df_filtered)} records with '{keyword}' in title")
    else:
        print("\n⏩ No filter applied")
    
    df_salaries = df_filtered

In [ ]:
#@title 📊 Statistics

if 'df_salaries' in dir() and len(df_salaries) > 0:
    print(f"\n📊 Salary Statistics for {state_name}\n")
    
    salaries = df_salaries['salary_numeric']
    salaries = salaries[salaries > 0]  # Remove zeros
    
    if len(salaries) > 0:
        print(f"Total employees:    {len(df_salaries):,}")
        print(f"Average salary:     ${salaries.mean():,.2f}")
        print(f"Median salary:      ${salaries.median():,.2f}")
        print(f"Highest salary:     ${salaries.max():,.2f}")
        print(f"Lowest salary:      ${salaries.min():,.2f}")
        print(f"Total payroll:      ${salaries.sum():,.2f}")
        
        print("\n\nTop employers by employee count:")
        print(df_salaries['employer'].value_counts().head(10).to_string())
    else:
        print("No valid salary data to analyze.")

In [ ]:
#@title 💾 Download Results

if 'df_salaries' in dir() and len(df_salaries) > 0:
    print("\n📥 Preparing downloads...\n")
    
    # Create filename
    timestamp = datetime.now().strftime('%Y%m%d')
    safe_term = re.sub(r'[^a-zA-Z0-9]', '_', search_term)[:30] if search_term else 'all'
    filename_base = f"{state_name}_Salaries_{safe_term}_{timestamp}"
    
    # Remove internal columns before export
    export_df = df_salaries.drop(columns=['salary_numeric'], errors='ignore')
    
    # Save as CSV
    csv_filename = f"{filename_base}.csv"
    export_df.to_csv(csv_filename, index=False)
    print(f"✅ Saved: {csv_filename}")
    
    # Save as Excel
    excel_filename = f"{filename_base}.xlsx"
    export_df.to_excel(excel_filename, index=False)
    print(f"✅ Saved: {excel_filename}")
    
    # Download files
    print("\n📥 Downloading files to your computer...")
    files.download(csv_filename)
    files.download(excel_filename)
    
    print("\n🎉 Done! Check your Downloads folder.")
else:
    print("\n⚠️ No data to download. Run the search cells above first.")

---

## 📊 Summary

You've extracted government salary data!

**What you got:**
- Employee names, titles, and departments
- Salary/compensation data
- Statistics and analysis
- CSV and Excel exports

**Tips:**
- Run again with different search terms
- Try different states for comparison
- Use the filter options to narrow results

---

### 🛠️ More Tools at [jimmytools.net](https://jimmytools.net)

Questions? [@JimmyToolsAi on X](https://x.com/JimmyToolsAi)
